In [1]:
import os 
%pwd


'c:\\Learning\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\\research'

In [9]:
'''os.chdir("../")'''
%pwd

'C:\\Learning\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [5]:
import os
os.chdir("C:/Learning/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC")
print("Working directory set to:", os.getcwd())
os.getcwd()

Working directory set to: C:\Learning\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC


'C:\\Learning\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

You need to make a token in dasghub then
in terminal write
dagshub login --token your token

 dagshub login --token 66ba938aeda0c5830ddc93215c7f0f744fe89f66

In [10]:
import tensorflow as tf  
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [14]:
from dagshub import dagshub_logger
import mlflow
import dagshub

# Initialize DagsHub connection (this uses the credentials saved from `dagshub login`)
dagshub.init(repo_owner='hamedmakian2018', 
             repo_name='End-to-End-Chest-Cancer-Classification-using-MLflow-DVC', 
             mlflow=True)
from mlflow.tracking import MlflowClient

experiment_name = "Chest-Cancer-Evaluation"
client = MlflowClient()

# Check if experiment already exists
experiment = client.get_experiment_by_name(experiment_name)

# If not, create and set it; otherwise, just set it
if experiment is None:
    mlflow.set_experiment(experiment_name)
else:
    mlflow.set_experiment(experiment_name)  # This is still needed to use it in the run

    
# Start an MLflow run
with mlflow.start_run():
    mlflow.log_param("learning_rate", 0.001)
    mlflow.log_metric("accuracy", 0.96)
    #mlflow.keras.log_model(model, artifact_path="VGG16_model")  # Logs with this name
    mlflow.tensorflow.log_model(model, artifact_path="VGG16_model")
    


Initialized MLflow to track repo "hamedmakian2018/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC"

Repository hamedmakian2018/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC initialized!

2025/05/05 23:28:25 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\hamed\AppData\Local\Temp\tmp_fk_uni_\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\hamed\AppData\Local\Temp\tmp_fk_uni_\model\data\model\assets


In [28]:
'''

from dagshub import dagshub_logger
import mlflow
import dagshub
import tensorflow as tf

# Your trained Keras model (make sure it's already defined or loaded)
model = tf.keras.models.load_model("artifacts/training/model.h5")

# Initialize DagsHub + MLflow
dagshub.init(
    repo_owner='hamedmakian2018', 
    repo_name='End-to-End-Chest-Cancer-Classification-using-MLflow-DVC', 
    mlflow=True
)

# Start logging
mlflow.set_experiment("Chest-Cancer-Eval")

with mlflow.start_run():
    mlflow.log_param("learning_rate", 0.001)
    mlflow.log_metric("accuracy", 0.96)
    mlflow.keras.log_model(model, artifact_path="VGG16_model")
'''

'\n\nfrom dagshub import dagshub_logger\nimport mlflow\nimport dagshub\nimport tensorflow as tf\n\n# Your trained Keras model (make sure it\'s already defined or loaded)\nmodel = tf.keras.models.load_model("artifacts/training/model.h5")\n\n# Initialize DagsHub + MLflow\ndagshub.init(\n    repo_owner=\'hamedmakian2018\', \n    repo_name=\'End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\', \n    mlflow=True\n)\n\n# Start logging\nmlflow.set_experiment("Chest-Cancer-Eval")\n\nwith mlflow.start_run():\n    mlflow.log_param("learning_rate", 0.001)\n    mlflow.log_metric("accuracy", 0.96)\n    mlflow.keras.log_model(model, artifact_path="VGG16_model")\n'

In [15]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int



In [16]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="End-to-End-Chest-Cancer-Classification-using-MLflow-DVC",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [17]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e


Found 102 images belonging to 2 classes.
7/7 [==============================] - 94s 13s/step - loss: 2.8359 - accuracy: 0.7255


2025/05/05 23:31:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\hamed\AppData\Local\Temp\tmp0enonpw5\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\hamed\AppData\Local\Temp\tmp0enonpw5\model\data\model\assets
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/05/05 23:32:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.


In [19]:
mlflow.set_tracking_uri("https://dagshub.com/hamedmakian2018/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC.mlflow")


In [20]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/hamedmakian2018/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC.mlflow")
mlflow.set_experiment("Test-Connection")

with mlflow.start_run():
    mlflow.log_param("test_param", 1)
    mlflow.log_metric("test_metric", 0.99)


2025/05/06 00:19:15 INFO mlflow.tracking.fluent: Experiment with name 'Test-Connection' does not exist. Creating a new experiment.


In [23]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_ids = [e.experiment_id for e in client.search_experiments()]
for eid in experiment_ids:
    exp = client.get_experiment(eid)
    print(f"Name: {exp.name}, ID: {exp.experiment_id}")


Name: Test-Connection, ID: 3
Name: Chest-Cancer-Evaluation, ID: 2


In [24]:
import mlflow
from mlflow.tracking import MlflowClient

# Set the DagsHub tracking URI
mlflow.set_tracking_uri("https://dagshub.com/hamedmakian2018/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC.mlflow")

# Create a client and try listing experiments
client = MlflowClient()
try:
    experiments = client.search_experiments()
    print("✅ Connection successful! Experiments found:")
    for exp in experiments:
        print(f"- {exp.name} (ID: {exp.experiment_id})")
except Exception as e:
    print("❌ Connection failed:", e)


✅ Connection successful! Experiments found:
- Chest-Cancer-Evaluation (ID: 2)
